# 知識補充工作坊 Python Google Places API

In [4]:
# 載入套件
import os
import json
import requests
import pandas as pd

## 1. Place Details API 串接

* 已知 Place ID ，直接蒐集地點細節資訊
* Request Method: Get
* Response: Google 定義之 Place 物件

In [5]:
my_key = "your_key_123456" # 請輸入個人的 API KEY

`requests.get()` 發送 HTTP GET request，回傳 requests 套件下 Class 為 `Response` 的物件

In [6]:
my_place_id = "ChIJL-BJs3aqQjQRT5fEKex6ruo" # 已知 Place ID
my_fields = "id,displayName,types,primaryType,location,rating,userRatingCount,dineIn,takeout" # 逗點分隔 # 不可有空格
my_languageCode = "zh-TW" # for Mandarin Chinese (Taiwan)

# 放入 place_id, fields, languageCode, key 等參數，發送 request
response = requests.get(f"https://places.googleapis.com/v1/places/{my_place_id}?fields={my_fields}&languageCode={my_languageCode}&key={my_key}")

In [7]:
type(response)

requests.models.Response

`Response.text` 取出 Response 中的文字內容

In [8]:
print(response.text)

{
  "id": "ChIJL-BJs3aqQjQRT5fEKex6ruo",
  "types": [
    "fast_food_restaurant",
    "hamburger_restaurant",
    "brunch_restaurant",
    "american_restaurant",
    "restaurant",
    "food",
    "point_of_interest",
    "establishment"
  ],
  "location": {
    "latitude": 24.9879391,
    "longitude": 121.5748269
  },
  "rating": 3,
  "userRatingCount": 1339,
  "displayName": {
    "text": "麥當勞-台北指南餐廳",
    "languageCode": "zh-TW"
  },
  "takeout": true,
  "dineIn": true,
  "primaryType": "fast_food_restaurant"
}



`Response.json()` 將 `Response` 中的 JSON 字串轉換成 Python 的 `Dictionary` 資料型態

In [9]:
place_details = response.json()
type(place_details)

dict

In [10]:
place_details

{'id': 'ChIJL-BJs3aqQjQRT5fEKex6ruo',
 'types': ['fast_food_restaurant',
  'hamburger_restaurant',
  'brunch_restaurant',
  'american_restaurant',
  'restaurant',
  'food',
  'point_of_interest',
  'establishment'],
 'location': {'latitude': 24.9879391, 'longitude': 121.5748269},
 'rating': 3,
 'userRatingCount': 1339,
 'displayName': {'text': '麥當勞-台北指南餐廳', 'languageCode': 'zh-TW'},
 'takeout': True,
 'dineIn': True,
 'primaryType': 'fast_food_restaurant'}

In [11]:
# 可用多個中括號取出所需資訊
{
    "id": place_details['id'],
    "name": place_details['displayName']['text'],
    "rating": place_details['rating'],
    "rating_count": place_details['userRatingCount'],
    "is_restaurant": "restaurant" in place_details['types'],
    "primary_type": place_details['primaryType'],
    "lat": place_details['location']['latitude'],
    "lng": place_details['location']['longitude'],
    "dine_in": place_details.get('dineIn', False),
    "takeout": place_details.get('takeout', False)
}

{'id': 'ChIJL-BJs3aqQjQRT5fEKex6ruo',
 'name': '麥當勞-台北指南餐廳',
 'rating': 3,
 'rating_count': 1339,
 'is_restaurant': True,
 'primary_type': 'fast_food_restaurant',
 'lat': 24.9879391,
 'lng': 121.5748269,
 'dine_in': True,
 'takeout': True}

## 2. 搜尋文字 API 串接

* 未知 Place ID，透過關鍵字搜尋，在指定空間範圍與特徵之下，找出符合要求的地點，並附上地點資訊
* Request Method: Post
* Response: JSON 物件，包含 places list 儲存多個 Place 物件

In [12]:
headers = {
    'X-Goog-Api-Key': my_key,
    'X-Goog-FieldMask': "places.id,places.displayName,places.types,places.primaryType,places.location,places.rating,places.userRatingCount,places.dineIn,places.takeout", # places.欄位名稱
}

data = {
    "textQuery": "咖啡廳",
    "maxResultCount": 10,
    "locationRestriction": {
        "rectangle": {
            "low": {
                "latitude": 25.01640963187937,
                "longitude": 121.52877784186686
            },
            "high": {
                "latitude": 25.02239049647823,
                "longitude": 121.534573876546
            }
        }
    },
    "languageCode": "zh-TW"
}

`json.dumps()` 將 POST 時欲提交的 data 由 `Dictionary` 轉換成 JSON 格式的字串，並處理編碼

In [13]:
json.dumps(data)

'{"textQuery": "\\u5496\\u5561\\u5ef3", "maxResultCount": 10, "locationRestriction": {"rectangle": {"low": {"latitude": 25.01640963187937, "longitude": 121.52877784186686}, "high": {"latitude": 25.02239049647823, "longitude": 121.534573876546}}}, "languageCode": "zh-TW"}'

`requests.post()` 發送 HTTP POST request，附上 data 以及 header

In [14]:
# 放入 Api-Key, FieldMask, textQuery, maxResultCount, locationBias, languageCode 等參數，發送 request
response = requests.post('https://places.googleapis.com/v1/places:searchText',
                         data=json.dumps(data),
                         headers=headers)

`Response.json()` 將 `Response` 中的 JSON 字串轉換成 Python 的 `Dictionary` 資料型態

In [15]:
response.json()

{'places': [{'id': 'ChIJffQZ6YipQjQRAdV3sCg2bcM',
   'types': ['coffee_shop',
    'cafe',
    'restaurant',
    'food',
    'point_of_interest',
    'store',
    'establishment'],
   'location': {'latitude': 25.021369000000004,
    'longitude': 121.53295240000001},
   'rating': 4.3,
   'userRatingCount': 714,
   'displayName': {'text': 'Rebirth Cafe & Restaurant',
    'languageCode': 'zh-TW'},
   'takeout': True,
   'dineIn': True,
   'primaryType': 'coffee_shop'},
  {'id': 'ChIJsygw_YipQjQR9OQ2KKEFI5Y',
   'types': ['cafe', 'food', 'point_of_interest', 'establishment'],
   'location': {'latitude': 25.0204797, 'longitude': 121.5335043},
   'rating': 4.7,
   'userRatingCount': 551,
   'displayName': {'text': '聞山咖啡台大店', 'languageCode': 'zh-TW'},
   'takeout': True,
   'dineIn': True,
   'primaryType': 'cafe'},
  {'id': 'ChIJiYZJcompQjQRoI4OnWDl3zk',
   'types': ['cafe',
    'coffee_shop',
    'food',
    'point_of_interest',
    'store',
    'establishment'],
   'location': {'latitude': 

In [16]:
# 取出 key 為 'places' 的值，為一包含多個 Place 物件的 list
places_list = response.json()['places']
places_list

[{'id': 'ChIJffQZ6YipQjQRAdV3sCg2bcM',
  'types': ['coffee_shop',
   'cafe',
   'restaurant',
   'food',
   'point_of_interest',
   'store',
   'establishment'],
  'location': {'latitude': 25.021369000000004,
   'longitude': 121.53295240000001},
  'rating': 4.3,
  'userRatingCount': 714,
  'displayName': {'text': 'Rebirth Cafe & Restaurant',
   'languageCode': 'zh-TW'},
  'takeout': True,
  'dineIn': True,
  'primaryType': 'coffee_shop'},
 {'id': 'ChIJsygw_YipQjQR9OQ2KKEFI5Y',
  'types': ['cafe', 'food', 'point_of_interest', 'establishment'],
  'location': {'latitude': 25.0204797, 'longitude': 121.5335043},
  'rating': 4.7,
  'userRatingCount': 551,
  'displayName': {'text': '聞山咖啡台大店', 'languageCode': 'zh-TW'},
  'takeout': True,
  'dineIn': True,
  'primaryType': 'cafe'},
 {'id': 'ChIJiYZJcompQjQRoI4OnWDl3zk',
  'types': ['cafe',
   'coffee_shop',
   'food',
   'point_of_interest',
   'store',
   'establishment'],
  'location': {'latitude': 25.0188775, 'longitude': 121.5334191},
  'ra

## 3. 解析 API 回應並儲存


提取每個地點中的所需資訊

In [17]:
# 建立一個空的 list
places = []

# 遍歷 places_list 中的每個 dictionary
for place in places_list:

    # 紀錄所需商家資訊
    place_info = {
        "id": place['id'],
        "name": place['displayName']['text'],
        "rating": place['rating'],
        "rating_count": place['userRatingCount'],
        "is_restaurant": "restaurant" in place['types'],
        "primary_type": place['primaryType'],
        "lat": place['location']['latitude'],
        "lng": place['location']['longitude'],
        "dine_in": place.get('dineIn', False),
        "takeout": place.get('takeout', False)
    }

    # 將記錄下來的資訊加到 list 中
    places.append(place_info)

places[:2]

[{'id': 'ChIJffQZ6YipQjQRAdV3sCg2bcM',
  'name': 'Rebirth Cafe & Restaurant',
  'rating': 4.3,
  'rating_count': 714,
  'is_restaurant': True,
  'primary_type': 'coffee_shop',
  'lat': 25.021369000000004,
  'lng': 121.53295240000001,
  'dine_in': True,
  'takeout': True},
 {'id': 'ChIJsygw_YipQjQR9OQ2KKEFI5Y',
  'name': '聞山咖啡台大店',
  'rating': 4.7,
  'rating_count': 551,
  'is_restaurant': False,
  'primary_type': 'cafe',
  'lat': 25.0204797,
  'lng': 121.5335043,
  'dine_in': True,
  'takeout': True}]

In [18]:
# 轉換成 pandas DataFrame 並輸出儲存
df = pd.DataFrame(places)
df

,id,name,rating,rating_count,is_restaurant,primary_type,lat,lng,dine_in,takeout
0,ChIJffQZ6YipQjQRAdV3sCg2bcM,Rebirth Cafe & Restaurant,4.3,714,True,coffee_shop,25.021369,121.532952,True,True
1,ChIJsygw_YipQjQR9OQ2KKEFI5Y,聞山咖啡台大店,4.7,551,False,cafe,25.020480,121.533504,True,True
2,ChIJiYZJcompQjQRoI4OnWDl3zk,靈感咖啡 Vegan Coffee Shop,4.6,392,False,cafe,25.018877,121.533419,True,True
3,ChIJN1y6RacCaDQRElenCbyeN2w,墨爾本布蕾斯咖啡廳 Melbourne Press Cafe | Brunch | Coff...,4.3,392,True,cafe,25.020478,121.533518,True,True
4,ChIJ4yq9ZImpQjQRRJQu4sJSdwY,西雅圖極品咖啡 台大店,4.0,789,False,coffee_shop,25.018256,121.533368,True,True
5,ChIJCRed4JmpQjQRpOEwyWt1wwU,達文西咖啡 - 台大旗艦店（溫州店）,4.6,503,False,cafe,25.020629,121.532941,True,True
6,ChIJ9ZdOiYipQjQRPPhSsgTk8J0,葛樂蒂咖啡館,4.4,879,False,cafe,25.022071,121.533897,True,True
7,ChIJAYjI7oipQjQRmqGtNWITDZU,AGCT apartment,4.4,636,False,cafe,25.021349,121.533145,True,True
8,ChIJ-09J7zupQjQRpE6bXqwDoQY,羊跳蚤,4.1,128,False,coffee_shop,25.021358,121.533147,True,True
9,ChIJ54r5aYmpQjQRSPx1fqonWXc,雪可屋,4.2,466,False,cafe,25.019601,121.532394,True,True


In [19]:
# 儲存至指定路徑
current_directory = os.getcwd()
df.to_csv(f"{current_directory}/../output/cafe_details.csv", index=False)